# A study on the monthly and annual variability of the Greenland (GrIS) and Antarctic Ice Sheet (AIS) gravimetric mass balance (GMB) and its trend over time.

## Import packages

In [ ]:
import calendar

import matplotlib.pyplot as plt
import pandas as pd
import pwlf
import scipy.stats
import xarray as xr
from c3s_eqc_automatic_quality_control import download

plt.style.use("seaborn-v0_8-notebook")

## Define Parameters

In [ ]:
variables = ["GrIS_total", "AntIS_total"]

## Define request

In [ ]:
collection_id = "satellite-ice-sheet-mass-balance"
request = {
    "variable": "all",
    "format": "zip",
}

## Download data

In [ ]:
ds = download.download_and_transform(collection_id, request)
ds_err = ds[[f"{var}_er" for var in variables]].compute()
ds = ds[variables].compute()

## Compute cumulative and non-cumulative mass change

In [ ]:
year_to_ns = 1.0e9 * 60 * 60 * 24 * 365
with xr.set_options(keep_attrs=True):
    ds = ds - ds.isel(time=0)
for var, da in ds.data_vars.items():
    da.attrs["region"] = da.attrs["long_name"].split("_", 1)[0].title()
    da.attrs["long_name"] = "Ice Mass Change"
for var, da in ds_err.data_vars.items():
    da.attrs["units"] += "/month"
    da.attrs["region"] = da.attrs["long_name"].split("_", 1)[0].title()
    da.attrs["long_name"] = "Ice Mass Error"

with xr.set_options(keep_attrs=True):
    ds_diff = ds.diff("time") / ds["time"].diff("time").astype(int)
    ds_diff *= year_to_ns / 12
for da in ds_diff.data_vars.values():
    da.attrs["long_name"] = da.attrs["long_name"].replace("Ice ", "")
    da.attrs["units"] += "/month"

## Find missing months

In [ ]:
start, end = ds["time"].isel(time=[0, -1]).dt.strftime("%d/%m/%Y").values.tolist()

for string, date in zip(("start", "end"), (start, end)):
    print(f"The {string:^5} date of the time series is", date)

expected = len(pd.date_range(start, end, freq="M", inclusive="both"))
actual = len(set(ds["time"].dt.strftime("%Y%m").values))
for string, date in zip(("expected", "present"), (expected, actual)):
    print(f"The amount of months that is {string} between these two dates is", date)

missing = 100 * abs(expected - actual) / expected
print(f"The amount of missing months is {missing:.2f}%")

(12 * ds["time"].diff("time").astype(int) / year_to_ns).plot()
plt.ylabel("Time gap [month]")
plt.grid()

## Define plotting function

In [ ]:
def plot_timeseries(ds):
    fig, axs = plt.subplots(len(variables), 1, layout="constrained")
    for ax, da in zip(axs, ds.data_vars.values()):
        da.plot(ax=ax, marker=".")
        ax.set_title(f"{da.attrs['region']} Ice Sheet")
        ax.grid()
    return fig, axs

## Plot cumulative timeseries

In [ ]:
fig, axs = plot_timeseries(ds)
_ = fig.suptitle("Cumulative mass change of the ice sheets from GRACE(-FO).")

## Plot non-cumulative timeseries

In [ ]:
fig, axs = plot_timeseries(ds_diff)
_ = fig.suptitle("Non-cumulative monthly ice mass changes of the ice sheets.")

## Plot error

In [ ]:
fig, axs = plot_timeseries(ds_err)
_ = fig.suptitle("Error estimate of the mass change of the ice sheets from GRACE(-FO).")

## Plot errorbar

In [ ]:
fig, axs = plt.subplots(len(variables), 1, layout="constrained")
for ax, da in zip(axs, ds.data_vars.values()):
    yerr = ds_err[f"{da.name}_er"]
    total_err = (yerr**2).sum() ** (1 / 2)
    print(f"{da.attrs['region']}:")
    print(
        "\tThe average mass change error of the Ice Sheet is "
        f"{yerr.mean().values:.2f} {yerr.attrs['units']}."
    )
    print(
        "\tThe cumulative mass change of the Ice Sheet is "
        f"{da.dropna('time')[-1].values:.2f} ± {total_err.values:.2f} {da.attrs['units']}."
    )
    da.to_pandas().plot(
        ax=ax,
        yerr=yerr,
        ecolor="k",
        elinewidth=0.25,
        capsize=2,
        capthick=1,
        grid=True,
        title=f"{da.attrs['region']} Ice Sheet",
    )
    ax.set_ylabel(f"{da.attrs['long_name']} [{da.attrs['units']}]")
_ = fig.suptitle("Cumulative mass change of the ice sheets from GRACE(-FO).")

## Plot montly boxplots

In [ ]:
fig, axs = plt.subplots(len(variables), 1, layout="constrained")
for ax, da in zip(axs, ds_diff.data_vars.values()):
    df = da.to_dataframe()
    df["month"] = df.index.month
    df.boxplot(
        by="month",
        ax=ax,
        ylabel=f"{da.attrs['long_name']} [{da.attrs['units']}]",
        xlabel="Month",
    )
    ax.set_title(f"{da.attrs['region']} Ice Sheet")
    ax.set_xticklabels([calendar.month_abbr[m] for m in ax.get_xticks()])
_ = fig.suptitle("Monthly ice mass changes of the ice sheets from GRACE(-FO).")

## Compute and plot piecewise trends

In [ ]:
fig, axs = plt.subplots(len(variables), 1, layout="constrained")
for ax, da, n_segments in zip(axs, ds.data_vars.values(), (4, 2)):
    da = da.dropna("time")
    x_hat = pd.date_range(
        *da["time"].isel(time=[0, -1]).values.tolist(), inclusive="both"
    )
    my_pwlf = pwlf.PiecewiseLinFit(da["time"].values.astype(int), da)
    breaks = my_pwlf.fit(n_segments)
    y_hat = my_pwlf.predict(x_hat.values.astype(int))
    da.plot(ax=ax, label="Data")
    ax.plot(x_hat, y_hat, label="Trend")
    ax.set_title(f"{da.attrs['region']} Ice Sheet")
    ax.grid()
    ax.legend()
_ = fig.suptitle(
    "Cumulative mass change of the ice sheets from GRACE(-FO) and its trends."
)

## Compute and plot trends

In [ ]:
fig, axs = plt.subplots(len(variables), 1, layout="constrained")
for ax, da in zip(axs, ds.data_vars.values()):
    da = da.dropna("time")
    print(da.attrs["region"] + ":")
    for label, degree in zip(
        (
            "Linear",
            "Quadratic",
        ),
        (1, 2),
    ):
        # Compute coefficients
        coeffs = da.polyfit("time", degree)["polyfit_coefficients"]

        # Plot trends and print stats
        equation = []
        for deg, coeff in coeffs.groupby("degree"):
            coeff = coeff.squeeze() * (year_to_ns**deg)
            if deg == degree:
                if deg == 1:
                    quantity = "slope"
                    units = "Gt/yr"
                elif deg == 2:
                    quantity = "acceleration"
                    units = "Gt/yr^2"
                else:
                    raise ValueError(f"{deg=}")
                print(
                    f"\tThe {quantity} of the Ice Sheet mass change is {degree*coeff:.3f} {units}."
                )
                if deg == 1:
                    _, p_value = scipy.stats.kendalltau(da["time"], da)
                    significance_level = 0.05
                    is_significant = p_value < significance_level
                    print(
                        " ".join(
                            [
                                "\tThe trend",
                                "is significant"
                                if is_significant
                                else "is not significant",
                                f"at an alpha level of {significance_level}, i.e. a monotonic trend",
                                "is present." if is_significant else "is not present.",
                            ]
                        )
                    )
            equation.append(
                f"{float(coeff):+.3f}{'x' if deg else ''}{f'$^{deg}$' if deg>1 else ''}"
            )
        label = f"{label}: {' '.join(equation[::-1])}"
        fit = xr.polyval(da["time"], coeffs)
        fit.plot(label=label, ax=ax)
    da.plot(label="Data", ax=ax)
    ax.set_title(f"{da.attrs['region']} Ice Sheet")
    ax.grid()
    ax.legend()
_ = fig.suptitle(
    "Cumulative mass change of the Greenland and Antarctic Ice Sheet and their trends."
)